### Hybrid Search LangChain

In [14]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [15]:
import os

# Set your Pinecone API key (replace with your actual key)
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [17]:
!pip install langchain-community


  Using cached PyYAML-6.0.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.31-cp310-cp310-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.2.9-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_core-0.2.20-py3-none-any.whl.metadata (6.0 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-macosx_10_9_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (31 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached marshmallow-3.21

In [18]:
from langchain_community.retrievers.pinecone_hybrid_search import PineconeHybridSearchRetriever



In [41]:
import os
from pinecone import Pinecone, ServerlessSpec
api_key="YOUR_API_KEY"
index_name="hybrid-search-langchain"
#initialize the Pinecone client
pc=Pinecone(api_key=api_key)

In [42]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [43]:
index=pc.Index(index_name)

In [44]:
index

In [32]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.0.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached huggingface_hub-0.23.5-py3-none-any.whl.metadata (12 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-macosx_10_12_x86_64.whl.metadata (6.7 kB)
  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached torch-2.2.2-cp310-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached scikit_learn-1.5.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached pillow-10.4.0-cp310-cp310-macosx_10_10_x86_64.whl.metadata (9.2 kB)
  Using cached safetensors-0.4.3-cp310-cp310-macosx_10_12_x86_64.whl.metadata (3.8 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using c

In [45]:
#Vector embedding and sparse matrix
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings= HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [46]:
from pinecone_text.sparse import BM25Encoder

bm25encoder=BM25Encoder().default()
bm25encoder

In [52]:
sentences=[
    "Climate change is one of the most pressing issues facing humanity today. Its effects are widespread, impacting various aspects of the environment and human society. One of the critical areas where climate change has a profound impact is global agriculture. This text explores the multifaceted effects of climate change on agriculture, including changes in crop yields, shifts in agricultural zones, and the socio-economic implications for farming communities.",
    "Changes in Crop Yields: Climate change influences crop yields through alterations in temperature, precipitation patterns, and the increased frequency of extreme weather events. Higher temperatures can accelerate the growth cycles of some crops, leading to earlier harvests but potentially lower yields. Conversely, increased levels of carbon dioxide (CO2) can enhance photosynthesis and growth in certain plants, although this benefit is often offset by the negative effects of heat stress and water scarcity.",
    "Shifts in Agricultural Zones As global temperatures rise, agricultural zones are shifting. Crops that were traditionally grown in temperate regions are now being cultivated in areas previously unsuitable for agriculture due to cooler climates. For instance, vineyards are now being established in regions of Northern Europe that were once considered too cold for grape production. However, this shift also means that regions currently suitable for specific crops may become less viable, forcing farmers to adapt by changing crops or altering farming practices.",
    "Socio-Economic Implications The socio-economic implications of climate change on agriculture are profound. Smallholder farmers, particularly in developing countries, are among the most vulnerable. They often lack the resources and technology needed to adapt to changing conditions. This vulnerability can lead to reduced income, food insecurity, and increased poverty. Additionally, changes in agricultural productivity can affect global food prices, influencing both the availability and affordability of food worldwide."

]
##tfidf
bm25encoder.fit(sentences)

#store the values to json file
bm25encoder.dump("bm25_values.json")

100%|██████████| 4/4 [00:00<00:00, 396.58it/s]


In [53]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25encoder,index=index)

In [54]:
retriever.add_texts([
    "Climate change is one of the most pressing issues facing humanity today. Its effects are widespread, impacting various aspects of the environment and human society. One of the critical areas where climate change has a profound impact is global agriculture. This text explores the multifaceted effects of climate change on agriculture, including changes in crop yields, shifts in agricultural zones, and the socio-economic implications for farming communities.",
    "Changes in Crop Yields: Climate change influences crop yields through alterations in temperature, precipitation patterns, and the increased frequency of extreme weather events. Higher temperatures can accelerate the growth cycles of some crops, leading to earlier harvests but potentially lower yields. Conversely, increased levels of carbon dioxide (CO2) can enhance photosynthesis and growth in certain plants, although this benefit is often offset by the negative effects of heat stress and water scarcity.",
    "Shifts in Agricultural Zones As global temperatures rise, agricultural zones are shifting. Crops that were traditionally grown in temperate regions are now being cultivated in areas previously unsuitable for agriculture due to cooler climates. For instance, vineyards are now being established in regions of Northern Europe that were once considered too cold for grape production. However, this shift also means that regions currently suitable for specific crops may become less viable, forcing farmers to adapt by changing crops or altering farming practices.",
    "Socio-Economic Implications The socio-economic implications of climate change on agriculture are profound. Smallholder farmers, particularly in developing countries, are among the most vulnerable. They often lack the resources and technology needed to adapt to changing conditions. This vulnerability can lead to reduced income, food insecurity, and increased poverty. Additionally, changes in agricultural productivity can affect global food prices, influencing both the availability and affordability of food worldwide."

])

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


In [57]:
result =retriever.invoke("How are agricultural zones shifting due to climate change")
result[0]

Document(page_content='Shifts in Agricultural Zones As global temperatures rise, agricultural zones are shifting. Crops that were traditionally grown in temperate regions are now being cultivated in areas previously unsuitable for agriculture due to cooler climates. For instance, vineyards are now being established in regions of Northern Europe that were once considered too cold for grape production. However, this shift also means that regions currently suitable for specific crops may become less viable, forcing farmers to adapt by changing crops or altering farming practices.')

In [58]:
result=retriever.invoke("What are the socio-economic implications of climate change on agriculture?")
result[0]

Document(page_content='Socio-Economic Implications The socio-economic implications of climate change on agriculture are profound. Smallholder farmers, particularly in developing countries, are among the most vulnerable. They often lack the resources and technology needed to adapt to changing conditions. This vulnerability can lead to reduced income, food insecurity, and increased poverty. Additionally, changes in agricultural productivity can affect global food prices, influencing both the availability and affordability of food worldwide.')